### Loading Environmental Dependencies

Initial uncertainty as to the amount of work being done in this notebook lead to a potentially larger-than-necessary number of dependencies being loaded in

In [ ]:
# Declaring imports
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv

Similarly, both potential Census-related API's were loaded as a precaution.

In [ ]:
# US Census Data requirements
from census import Census
from us import states

### Establishing API Key

If being run, be sure to include your US Census API Key in your `.env`

In [ ]:
# API Key
# Load environment variables and U.S. Census API key
load_dotenv()
api_key = os.getenv("CENSUS_API_KEY")

### API Request Loop

Built off the code we learned for Module 6, Day 3, Exercise 3, this loop runs through the `.get()` requests for the years 2012 to 2022 and assigns a year value to DataFrame created from the returned `.json`s

In [ ]:
# Create empty list for the eventual DataFrame
acs5_list = []

# Create a list of years for the Census Library instance
years = [x for x in range(2012,2023)]

# For statement to loop through years in `years` as instances for Census Library
for year in years:
    # Create an instance of the Census library
    c = Census(
        api_key,
        year = int(year)
    )
    # Run Census Search to retrieve data on all states (ACS1 Census for `year`)
    acs5_data = c.acs5.get(
        (
            "NAME",
            "B19013_001E", # income
            "B19301_001E", # income_per_capita
            "B23025_002E", # employment_labor_force
            "B23025_003E", # employment_civilian_labor_force
            "B23025_004E", # employment_employed
            "B23025_005E", # employment_unemployed
            "C24010_006E", # employment_male_business_and_financial_operations_occupations
            "C24010_007E" , # employment_male_computer_engineering_and_science_occupations
            "C24010_008E" , # employment_male_computer_and_mathematical_occupations
            "C24010_042E" , # employment_female_business_and_financial_operations_occupations
            "C24010_043E" , # employment_female_computer_engineering_and_science_occupations
            "C24010_044E" , # employment_female_computer_and_mathematical_occupations
            "B17001_002E" , # poverty
            "B01002_001E" , # age
            "B01003_001E" , # population
            "B25064_001E" , # median_gross_rent
            "B25077_001E" , # median_home_value
            "B15003_002E" , # education_none
            "B15003_017E" , # education_high_school
            "B15003_018E" , # education_ged
            "B15003_021E" , # education_associates
            "B15003_022E" , # education_bachelors
            "B15003_023E" , # education_masters
            "B15003_024E" , # education_professional
            "B15003_025E" # education_doctorate
        ), 
        {'for': 'state:*'}
    )
    # Convert to DataFrame
    acs5_part = pd.DataFrame(acs5_data)
    # Add `year` to DataFrame
    acs5_part['Year'] = year
    # Append data from `year` to working DataFrame
    acs5_list.append(acs5_part)

### Data Cleaning

As a list of DataFrames was created with the previous loop, `.concat()` is necessary to group all the data on one DataFrame

In [ ]:
# Concatenating the list of DataFrames into one DataFrame
combined_acs5 = pd.concat(acs5_list, ignore_index=True)

Columns are then renamed for readability

In [ ]:
# Column renaming
combined_acs5 = combined_acs5.rename(
    columns = {
        "NAME": "Name",
        "B19013_001E": "income",
        "B19301_001E": "income_per_capita",
        "B23025_002E": "employment_labor_force",
        "B23025_003E": "employment_civilian_labor_force",
        "B23025_004E": "employment_employed",
        "B23025_005E": "employment_unemployed",
        "C24010_006E": "employment_male_business_and_financial_operations_occupations",
        "C24010_007E": "employment_male_computer_engineering_and_science_occupations",
        "C24010_008E": "employment_male_computer_and_mathematical_occupations",
        "C24010_042E": "employment_female_business_and_financial_operations_occupations",
        "C24010_043E": "employment_female_computer_engineering_and_science_occupations",
        "C24010_044E": "employment_female_computer_and_mathematical_occupations",
        "B17001_002E": "poverty",
        "B01002_001E": "age",
        "B01003_001E": "population",
        "B25064_001E": "median_gross_rent",
        "B25077_001E": "median_home_value",
        "B15003_002E": "education_none",
        "B15003_017E": "education_high_school",
        "B15003_018E": "education_ged",
        "B15003_021E": "education_associates",
        "B15003_022E": "education_bachelors",
        "B15003_023E": "education_masters",
        "B15003_024E": "education_professional",
        "B15003_025E": "education_doctorate"
    }
)

Final confirmation that the `Year` was applied correctly during previous steps, and that all requested years are in the final DataFrame

In [ ]:
# Confirming the concatenation
display(combined_acs5.head(3))
display(combined_acs5.tail(3))

### Export

Once cleaned and prepared, the DataFrame is exported as a `.csv` for use in the group `.ipynb` file

In [ ]:
# Explorting to `.csv` for use in group
combined_acs5.to_csv('combined_acs5_12-22_data.csv')